# HashiCorp Vault Demo for Transit Engine and Advanced Data Protection (ADP) features.

This demo shows how HashiCorp Vault Transit engine encryption and ADP features work.  ADP features include Format Preserving Encryption (FPE), Data Masking, and Tokenization.

To run this notebook in VS Code, chose the Jupyter kernel and then Bash

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

This also assumes you have the Vault CLI installed.

You will need Vault to be installed with an ADP Transform license add-on.

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-v $LOGS_PATH:/vault_logs \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# Vault Transit Engine
This section demonstrates the base encryption/decryption functions.

Ref: https://developer.hashicorp.com/vault/docs/secrets/transit

In [ ]:
# Enable transit engine at the default mount path
#vault secrets disable transit
vault secrets enable transit

In [ ]:
# Create a key
vault write -f transit/keys/demo-key

# Policy sample to use key
# Ref: https://developer.hashicorp.com/vault/tutorials/encryption-as-a-service/eaas-transit?variants=vault-deploy%3Aselfhosted#create-a-token-for-vault-clients

In [ ]:
# Encrypt plain text with key and store cipher text
# Before encrypting, value will be base64 encoded
vault write -f transit/encrypt/demo-key plaintext=$(base64 <<< "4111 1111 1111 1111")

# Store cipher text
export CIPHERTEXT=$(vault write -f -field=ciphertext transit/encrypt/demo-key plaintext=$(base64 <<< "4111 1111 1111 1111"))
echo
echo "Encrypted value: $CIPHERTEXT"

In [ ]:
# Decrypt cipher text
echo "Encrypted value: $CIPHERTEXT"
echo
echo "Decrypted value: $(base64 --decode <<< $(vault write -field=plaintext transit/decrypt/demo-key ciphertext=$CIPHERTEXT))"


In [ ]:
# Demo key rotation
vault write -f transit/keys/demo-key/rotate

In [ ]:
# Show that the key_version is now incremented
vault write -f transit/encrypt/demo-key plaintext=$(base64 <<< "4111 1111 1111 1111")

In [ ]:
# Show that you cannot export a key if it is not marked exportable
vault read transit/export/encryption-key/demo-key

# Show that you cannot backup the key
vault read transit/backup/demo-key

# Show that you cannot delete a key if it is not marked as deletion allowed
vault delete transit/keys/demo-key

In [ ]:
# Let's create a new key to show export and deletion capability

# Mark as exportable on creation
vault write -f transit/keys/demo-key2 exportable=true

In [ ]:
# Configure allow deletion and allows backup/restore in plaintext format
vault write transit/keys/demo-key2/config deletion_allowed=true allow_plaintext_backup=true

In [ ]:
# Show that you can now export the key
vault read transit/export/encryption-key/demo-key2
echo
# Show that you can do a backup of the key
vault read transit/backup/demo-key2
export KEY_BACKUP=$(vault read -field=backup transit/backup/demo-key2)

In [ ]:
# Show that you can delete the key now
vault delete transit/keys/demo-key2

# Show that the key is deleted
vault read transit/export/encryption-key/demo-key2

In [ ]:
# Let's restore the key now
echo "Key Backup: $KEY_BACKUP"
# Restore the key
vault write transit/restore/demo-key2 backup=$KEY_BACKUP
echo
# Show that the key is restored
vault read transit/export/encryption-key/demo-key2

## Setup Vault's Transform Engine (ADP Feature)

In [ ]:
# Enable transform engine at the default mount path
# Policy for configuration - https://developer.hashicorp.com/vault/tutorials/adp/transform#policy-requirements
vault secrets enable transform

# Format Preserving Encryption (FPE) Demo - Credit Card Number

In [ ]:
# Create a role named "payments" with "card-number" transformation attached
vault write transform/role/payments transformations=card-number

In [ ]:
# Verify the role "payments" is created
vault list transform/role

In [ ]:
# View existing templates.  Show that we will be using "builtin/creditcardnumber"
vault list transform/template

In [ ]:
# Create a transformation named "card-number" which will be used by role "payments" to transform credit card numbers.
# This uses the built-in template builtin/creditcardnumber to perform format-preserving encryption (FPE). 
vault write transform/transformations/fpe/card-number \
    template="builtin/creditcardnumber" \
    tweak_source=internal \
    allowed_roles=payments

In [ ]:
# Verify the "card-number" transformation is created
vault list transform/transformations/fpe

In [ ]:
# View the "card-number" transformation details
# https://developer.hashicorp.com/vault/tutorials/adp/transform#policy-requirements
vault read transform/transformations/fpe/card-number

In [ ]:
# Encode credit card number using FPE
vault write transform/encode/payments value=1111-2222-3333-4444 transformation=card-number

# Store encoded value
export FPE_ENCODED_VALUE=$(vault write -format json transform/encode/payments value=1111-2222-3333-4444 \
  transformation=card-number | jq -r .data.encoded_value)

In [ ]:
# Decode credit card number
vault write transform/decode/payments value=$FPE_ENCODED_VALUE transformation=card-number

# FPE Demo - NRIC

In [ ]:
# Create template for Singapore NRIC.  Note that we are not changing the front and ending letters.  
vault write transform/template/sg-nric-tmpl \
    type=regex \
    pattern="[stfgSTFG](\d{7})[a-zA-Z]" \
    alphabet=builtin/numeric 

In [ ]:
# View added template
vault list transform/template

In [ ]:
# Create transformation named "sg-nric" with the "sg-nric-tmpl" template
vault write transform/transformations/fpe/sg-nric \
    template=sg-nric-tmpl \
    tweak_source=internal \
    allowed_roles=hr-role


In [ ]:
# Create a role named "hr-role" with "sg-nric" transformation attached
vault write transform/role/hr-role transformations=sg-nric

In [ ]:
# Encode a NRIC value
vault write transform/encode/hr-role value="S7012345Z" transformation=sg-nric

export FPE_ENCODED_VALUE=$(vault write -format json transform/encode/hr-role value="S7012345Z" transformation=sg-nric | jq -r .data.encoded_value)

In [ ]:
# Decode NRIC
echo -e "FPE Encoded Value is: $FPE_ENCODED_VALUE\n"
vault write transform/decode/hr-role value=$FPE_ENCODED_VALUE transformation=sg-nric

# Data Masking Demo - IP Address

In [ ]:
# Create template for IP address.  I want to mask all digits.
vault write transform/template/ip-address-tmpl type=regex \
    pattern="(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])\.(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])\.(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])\.(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])" \
    alphabet=builtin/numeric

In [ ]:
# View added template
vault list transform/template

In [ ]:
# Create a transformation named "ip-address" which will be used by role "logging-role" 
vault write transform/transformations/masking/ip-address \
    template=ip-address-tmpl \
    masking_character=# \
    allowed_roles='logging-role'


In [ ]:
# Create a role named "logging-role" with "ip-address" transformation attached
vault write transform/role/logging-role transformations=ip-address



In [ ]:
# Mask a valid IP address
vault write transform/encode/logging-role value="10.100.1.87" \
    transformation=ip-address

# Tokenization Demo - Credit Card Number

In [ ]:
# Run a PostgreSQL database for tokenization demo
docker run --name postgres \
     -p 5432:5432 \
     --rm \
     -e POSTGRES_USER=root \
     -e POSTGRES_PASSWORD=mypassword \
     -d postgres

In [ ]:
# Verify Postgres database is running
docker ps

In [ ]:
# Create a role named "mobile-pay" with a transformation named "credit-card"
vault write transform/role/mobile-pay transformations=credit-card

In [ ]:
# Assumes your Vault is running on docker and connecting via the docker bridge network
# Obtain IP address of the postgres database for configuration
export POSTGRES_DB_IP=$(docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' postgres)
echo -e "Postgres IP Address is: $POSTGRES_DB_IP\n"
# Create a store which points to the postgres database
vault write transform/stores/postgres \
   type=sql \
   driver=postgres \
   supported_transformations=tokenization \
   connection_string="postgresql://{{username}}:{{password}}@$POSTGRES_DB_IP/root?sslmode=disable" \
   username=root \
   password=mypassword


In [ ]:
# Show that there is a new postgres store created
vault list transform/stores
echo
vault read transform/stores/postgres

In [ ]:
# Create a schema in postgres to store tokeniation artifacts
vault write transform/stores/postgres/schema transformation_type=tokenization \
    username=root password=mypassword

In [ ]:
# Create a transformation named "credit-card" which sets the generated token's max time-to-live (TTL) to 24 hours.
# Specify the postgres store
vault write transform/transformations/tokenization/credit-card \
  allowed_roles=mobile-pay \
  stores=postgres \
  max_ttl=24h


In [ ]:
# You can read back the properties of the transformation
vault read transform/transformations/tokenization/credit-card

In [ ]:
# Tokenize a credit card number.  Specify token time to live as 30 secs.
# Note policy to use tokenization - https://developer.hashicorp.com/vault/tutorials/adp/tokenization#tokenize-secrets

# vault write transform/encode/mobile-pay value=1111-2222-3333-4444 \
#      transformation=credit-card \
#      ttl=30s \
#      metadata="Organization=HashiCorp" \
#      metadata="Purpose=Travel" \
#      metadata="Type=AMEX"

export TOKEN_ENCODED_VALUE=$(vault write -format json transform/encode/mobile-pay value=1111-2222-3333-4444 \
     transformation=credit-card \
     ttl=30s \
     metadata="Organization=HashiCorp" \
     metadata="Purpose=Travel" \
     metadata="Type=AMEX" \
     | jq -r .data.encoded_value)
echo "encoded_value is : $TOKEN_ENCODED_VALUE"

# Note you can show the operation below within the 30s window and see the behaviour.

In [ ]:
# Retrieve metadata of the token
vault write transform/metadata/mobile-pay value=$TOKEN_ENCODED_VALUE transformation=credit-card

In [ ]:
# Validate the token value
vault write transform/validate/mobile-pay value=$TOKEN_ENCODED_VALUE transformation=credit-card

In [ ]:
# Validate if the credit card number has been tokenized already
vault write transform/tokenized/mobile-pay value=1111-2222-3333-4444 transformation=credit-card

In [ ]:
# Detokenize to retrieve the original plaintext credit card value

# vault write transform/decode/mobile-pay transformation=credit-card value=$TOKEN_ENCODED_VALUE

vault write -format json transform/decode/mobile-pay transformation=credit-card value=$TOKEN_ENCODED_VALUE | jq -r .data.decoded_value

# View Contents of PostgreSQL Database

In [ ]:
# Setup psql alias to the container
alias psql="docker exec -it postgres psql -U root"

# View tokens in the postgres store
psql -c '\x auto;' -c 'select * from tokens;'

# Cleanup

In [ ]:
# Cleanup
# stop PostgreSQL container
docker stop postgres

# Disable transit engine
vault secrets disable transit

# Disable transform engine
vault secrets disable transform

# Stop Vault container
docker stop vault-enterprise